# 📘 Flotorch Email Ticket Router Demo (CrewAI)

This notebook demonstrates how to automate email-based ticket routing using Flotorch CrewAI Agent and custom tools for Gmail.

## 🧭 Workflow Overview

1. **Fetch unread emails** from your inbox
2. **The agent** (configured in Flotorch Console) analyzes each email:
   - Detects intent (bug, feature, issue, etc.)
   - Determines routing team & priority
   - Summarizes the issue clearly
3. **Automatically routes** the task to the correct team lead
4. **Sends acknowledgment** to the sender confirming receipt

## 💡 Goal

Streamline internal task management from **"email → assignment"** in one autonomous flow.


# 📋 PRE-RUN CHECKLIST

Before you run this notebook, complete these setup steps:

## 1️⃣ Create a Flotorch Agent in Console

- **Agent name**: `support-ticket-inbox-router` (or match `AGENT_NAME` variable)
- **System Prompt**: 
      You are the Support Ticket Inbox Router. Workflow:

      1) Call fetch_mail_tool() - which returns a list of unread emails in the form:
         [{ "id": "...", "sender": "...", "subject": "...", "body": "..." }, ...]

      2) For each email returned:
         a) Analyze subject+body and produce a structured JSON with keys:
            - intent: one of ["bug","feature","task","issue","query"]
            - title: short developer-friendly title
            - summary: 1-3 sentence summary with reproduction steps if provided
            - routing_team: one of ["ui","python_backend","views_backend","devops","qa","technical"]
            - priority: one of ["P0 - Critical","P1 - High","P2 - Medium","P3 - Low","P4 - Info"]
            - recommended_sla: e.g. "Immediate (<30m)", "1 hour", "4 hours", "24 hours"
            - ack_message: short text to send to sender
            - notify_message: short text to send to lead

         b) Call route_task_tool(routing_team, title, summary, priority)
            - expect result: {"lead_name","lead_email","status"}

         c) Call send_ack_tool(sender, ack_message, ack_subject) to acknowledge sender.

      3) After processing all emails, return a summary JSON list of processed items with their tool-call statuses.

      Important: 
            - ONLY call the tools above. Do NOT send emails yourself — use send_ack_tool and route_task_tool to send messages.
            - NEVER describe tool usage in your response text. ALWAYS actually call the tools.
            - Do NOT provide a final response until ALL required tools have been called and executed.
            - For each email, you MUST call both route_task_tool AND send_ack_tool - do not skip any steps.
            - Continue the conversation loop until all tools are executed. Do not end with a summary that describes tool usage.
            **STRICTLY CALL ALL TOOLS - DO NOT DESCRIBE THEM IN TEXT.**      
- **Goal**: 
      To automatically manage and route internal task or issue requests received via email.
      When executed, this agent:
      1. Fetches new emails from a configured inbox using the fetch_mail tool.
      2. Analyzes each email’s subject and body to identify its intent (bug, feature, issue, or task), summarize the content, and determine the responsible team.
      3. Uses reasoning to decide the priority and urgency of each item.
      4. Invokes tools to route the task to the correct team lead (via email or future task system integration).
      5. Sends an acknowledgment reply to the original sender confirming the action taken.
      The agent ensures that communication between the sender and the responsible team is automated and consistent.

- **Model Settings**: add llm model
- **Save/Deploy**: Save and publish the agent

## 2️⃣ Create and Store Your Flotorch Credentials

In the notebook **Configuration** cell, set:

```python
FLOTORCH_API_KEY = "sk_..."
FLOTORCH_BASE_URL = "https://<your-gateway>.flotorch.cloud"
```

## 3️⃣ Prepare Gmail (App Password + IMAP)

### Steps:

1. **Sign in** to your Gmail account
2. **Enable 2-Step Verification** (if not already enabled)
3. **Create an App Password**:
   - Go to **Google Account → Security → App passwords**
   - Select **App**: Mail, **Device**: Other, name (e.g., "Notebook")
   - Copy the 16-character app password
4. **Enable IMAP** in Gmail settings:
   - Gmail → Settings → See all settings → Forwarding and POP/IMAP → **Enable IMAP**

### In the notebook Configuration cell, set:

```python
GMAIL_EMAIL = "your_email@gmail.com"
GMAIL_APP_PASSWORD = "16-char-app-password"
```

## 4️⃣ Customize Team Leads

In the notebook `TEAMS` dict, set correct `lead_emails` for your organization.

## refer last cell for sample examples do a test mail with that sample

In [ ]:
# install flotorch crewai package
%pip install flotorch[crewai]

In [ ]:
# ======================================================================
# 🧩 STEP 2 — Configuration
# ======================================================================
# Fill in your credentials directly below.
# This keeps the notebook self-contained and demo-ready.
# ======================================================================

FLOTORCH_API_KEY = "sk_"         # 🔑 your Flotorch API key
FLOTORCH_BASE_URL = "<gateway url>"   # 🌐 your Flotorch Gateway URL
AGENT_NAME = "<agent_name>"       # 🤖 Agent name configured in Console
APP_NAME = "email_ticket_router_demo"
USER_ID = "email_router_user_001"

# Gmail Credentials (create an App Password under Google Account -> Security -> App Passwords)
GMAIL_EMAIL = "<gmail>"
GMAIL_APP_PASSWORD = "<app password>"

# Team routing map (can customize for your org)
TEAMS = {
    "ui": {"lead_name": "<name>", "lead_email": "<gmail>"},
    "python_backend": {"lead_name": "<name>", "lead_email": "<gmail>"},
    "devops": {"lead_name": "<name>", "lead_email": "<gmail>"},
}

# Mail settings
IMAP_SERVER = "imap.gmail.com"
SMTP_SERVER = "smtp.gmail.com"
MAX_EMAILS_PER_RUN = 5
GMAIL_SUBJECT_KEYWORDS = ["bug", "issue", "feature", "task"]

print("✅ Configuration loaded successfully.")


In [ ]:
# ======================================================================
# 📬 STEP 3 — Gmail Utilities
# ======================================================================
# Functions for connecting to Gmail (IMAP & SMTP),
# fetching unread emails, and sending replies.
# ======================================================================

import imaplib, smtplib, email, re
from email.header import decode_header, make_header
from email.mime.text import MIMEText
from email.utils import parseaddr
from typing import List, Tuple

# --- IMAP Connection ---
def connect_imap():
    m = imaplib.IMAP4_SSL(IMAP_SERVER)
    m.login(GMAIL_EMAIL, GMAIL_APP_PASSWORD)
    return m

# --- Fetch unread emails matching keywords ---
def search_unseen_matching_subject(m, keywords: List[str], max_count: int):
    m.select("INBOX")
    status, data = m.search(None, 'UNSEEN')
    if status != 'OK':
        return []
    ids = data[0].split()
    results = []
    for msg_id in reversed(ids):
        if len(results) >= max_count:
            break
        status, msg_data = m.fetch(msg_id, '(RFC822)')
        if status != 'OK':
            continue
        msg = email.message_from_bytes(msg_data[0][1])
        subj_raw = msg.get('Subject', '')
        try:
            subj = str(make_header(decode_header(subj_raw)))
        except Exception:
            subj = subj_raw
        if any(k.lower() in subj.lower() for k in keywords):
            results.append((msg_id, msg))
    return results

# --- Extract sender, subject, body ---
def extract_sender_subject_body(msg) -> Tuple[str, str, str]:
    sender = parseaddr(msg.get('From', ''))[1]
    subject = str(make_header(decode_header(msg.get('Subject', ''))))
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain' and 'attachment' not in str(part.get("Content-Disposition", "")):
                body = part.get_payload(decode=True).decode(part.get_content_charset() or 'utf-8', errors='ignore')
                break
    else:
        payload = msg.get_payload(decode=True)
        if payload:
            body = payload.decode(msg.get_content_charset() or 'utf-8', errors='ignore')
    body = re.sub(r"^>+.*$", "", body, flags=re.MULTILINE).strip()
    return sender, subject, body

# --- SMTP Send Helper ---
def send_smtp_text(to_addr: str, subject: str, body: str):
    msg = MIMEText(body, _charset='utf-8')
    msg['From'] = GMAIL_EMAIL
    msg['To'] = to_addr
    msg['Subject'] = subject
    with smtplib.SMTP_SSL(SMTP_SERVER, 465) as s:
        s.login(GMAIL_EMAIL, GMAIL_APP_PASSWORD)
        s.sendmail(GMAIL_EMAIL, [to_addr], msg.as_string())

print("📨 Gmail utilities ready.")


In [ ]:
# ======================================================================
# 🛠️ STEP 4 — Define Custom Tools
# ======================================================================
# These tools are the interface between the agent and real-world actions.
# The agent decides *when and how* to call them.
# For CrewAI, we use the @tool decorator instead of FunctionTool.
# ======================================================================

from crewai.tools import tool
from typing import Dict, Optional
import json

# --- Tool A: Fetch unread emails ---
@tool
def fetch_mail_tool(max_results: int = MAX_EMAILS_PER_RUN) -> str:
    """
    Fetch unread emails from Gmail inbox matching keywords.
    
    IMPORTANT: This tool returns a JSON array of emails. Each email object contains:
    - id: unique email identifier
    - sender: email address of the sender (use this for send_ack_tool)
    - subject: email subject line
    - body: email body content
    
    You MUST use the actual "sender" field from each email when calling send_ack_tool.
    
    Args:
        max_results (int): Maximum number of emails to fetch (default: MAX_EMAILS_PER_RUN)
    
    Returns:
        str: JSON string containing list of emails with id, sender, subject, and body
        Example: [{"id": "123", "sender": "user@example.com", "subject": "...", "body": "..."}, ...]
    """
    import traceback
    print(f"🔧 [TOOL EXECUTION] fetch_mail_tool called with max_results={max_results}")
    print(f"   [DEBUG] Keywords to match: {GMAIL_SUBJECT_KEYWORDS}")
    try:
        print(f"   [DEBUG] Connecting to IMAP server...")
        m = connect_imap()
        print(f"   [DEBUG] Searching for unread emails...")
        matches = search_unseen_matching_subject(m, GMAIL_SUBJECT_KEYWORDS, max_results)
        print(f"   [DEBUG] Found {len(matches)} matching email(s)")
        emails = []
        for msg_id, msg in matches:
            sender, subject, body = extract_sender_subject_body(msg)
            email_data = {
                "id": msg_id.decode() if isinstance(msg_id, bytes) else str(msg_id),
                "sender": sender,
                "subject": subject,
                "body": body
            }
            emails.append(email_data)
            print(f"   [DEBUG] Processed email from {sender}, subject: {subject[:50]}...")
        m.logout()
        result = json.dumps(emails, indent=2)
        print(f"✅ [TOOL RESULT] fetch_mail_tool found {len(emails)} email(s)")
        if len(emails) == 0:
            print(f"   ⚠️  [WARNING] No emails found! Check if inbox has unread emails matching keywords: {GMAIL_SUBJECT_KEYWORDS}")
        else:
            print(f"   [DEBUG] Email senders: {[e['sender'] for e in emails]}")
        return result
    except Exception as e:
        error_msg = f"Error in fetch_mail_tool: {str(e)}"
        print(f"❌ [TOOL ERROR] {error_msg}")
        print(f"   [DEBUG] Traceback: {traceback.format_exc()}")
        return json.dumps({"error": error_msg, "emails": []})

# --- Tool B: Route the task to the appropriate team lead ---
@tool
def route_task_tool(routing_team: str, title: str, summary: str, priority: str) -> str:
    """
    Route a task to the appropriate team lead via email.
    
    IMPORTANT: This tool MUST be called for EVERY email that is processed.
    It sends a notification email to the team lead about the assigned task.
    
    Args:
        routing_team (str): Team name (ui, python_backend, views_backend, devops, qa, technical)
        title (str): Task title
        summary (str): Task summary
        priority (str): Priority level (P0 - Critical, P1 - High, P2 - Medium, P3 - Low, P4 - Info)
    
    Returns:
        str: JSON string with lead_name, lead_email, and status
    """
    import traceback
    print(f"🔧 [TOOL EXECUTION] route_task_tool called: team={routing_team}, title={title}, priority={priority}")
    print(f"   [DEBUG] routing_team type: {type(routing_team)}, value: {repr(routing_team)}")
    try:
        team_info = TEAMS.get(routing_team.lower())
        if not team_info:
            error_result = json.dumps({"status": f"error: unknown team '{routing_team}'", "available_teams": list(TEAMS.keys())})
            print(f"❌ [TOOL ERROR] Unknown team: {routing_team}")
            print(f"   [DEBUG] Available teams: {list(TEAMS.keys())}")
            return error_result
        
        print(f"   [DEBUG] Team info found: {team_info}")
        subject = f"[{priority}] {title}"
        body = f"Hi {team_info['lead_name']},\n\nAssigned: {title}\nPriority: {priority}\n\nSummary:\n{summary}\n\n— Task Router"
        
        print(f"   [DEBUG] Attempting to send email to {team_info['lead_email']}")
        send_smtp_text(team_info["lead_email"], subject, body)
        print(f"   [DEBUG] Email sent successfully")
        
        result = {"lead_name": team_info["lead_name"], "lead_email": team_info["lead_email"], "status": "sent"}
        print(f"✅ [TOOL RESULT] route_task_tool sent email to {team_info['lead_email']}")
        return json.dumps(result)
    except Exception as e:
        error_msg = f"Error in route_task_tool: {str(e)}"
        print(f"❌ [TOOL ERROR] {error_msg}")
        print(f"   [DEBUG] Traceback: {traceback.format_exc()}")
        return json.dumps({"error": error_msg, "status": "failed"})

# --- Tool C: Send acknowledgment to sender ---
@tool
def send_ack_tool(sender_email: str, ack_body: str, ack_subject: str = "Acknowledgment: we've received your request") -> str:
    """
    Send an acknowledgment email to the original sender.
    
    CRITICAL: Use the ACTUAL sender email address from the email object returned by fetch_mail_tool.
    DO NOT use placeholder emails like "example@example.com" or "test@test.com".
    The sender_email must be the real email address from the fetched email's "sender" field.
    
    IMPORTANT: This tool MUST be called AFTER route_task_tool for each email.
    You MUST call this tool for EACH email that was processed.
    
    Args:
        sender_email (str): Email address of the sender (MUST be from the email object's "sender" field)
        ack_body (str): Acknowledgment message body
        ack_subject (str): Acknowledgment email subject (default: "Acknowledgment: we've received your request")
    
    Returns:
        str: JSON string with recipient email and status. Example: {"to": "sender@example.com", "status": "sent"}
    """
    import traceback
    import sys
    
    # Ensure sender_email is a string
    if not isinstance(sender_email, str):
        sender_email = str(sender_email)
    
    # Ensure ack_subject is a string
    if not isinstance(ack_subject, str):
        ack_subject = str(ack_subject) if ack_subject else "Acknowledgment: we've received your request"
    
    # Validate that it's not a placeholder email
    if "example" in sender_email.lower() or "test" in sender_email.lower() or "@" not in sender_email:
        error_msg = f"Invalid sender email: {sender_email}. Must use actual sender email from fetched email object."
        print(f"❌ [TOOL ERROR] {error_msg}", flush=True)
        return json.dumps({"error": error_msg, "status": "failed"})
    
    try:
        # Use provided subject or default
        if not ack_subject or ack_subject.strip() == "":
            ack_subject = "Acknowledgment: we've received your request"
        
        print(f"   [DEBUG] Attempting to send acknowledgment email to {sender_email}...", flush=True)
        send_smtp_text(sender_email, ack_subject, ack_body)
        print(f"   [DEBUG] Acknowledgment email sent successfully", flush=True)
        
        result = {"to": sender_email, "status": "sent"}
        print(f"✅ [TOOL RESULT] send_ack_tool sent acknowledgment to {sender_email}", flush=True)
        return json.dumps(result)
    except Exception as e:
        error_msg = f"Error in send_ack_tool: {str(e)}"
        print(f"❌ [TOOL ERROR] {error_msg}", flush=True)
        print(f"   [DEBUG] Traceback: {traceback.format_exc()}", flush=True)
        return json.dumps({"error": error_msg, "status": "failed"})

# List of custom tools to pass to the agent
custom_tools = [fetch_mail_tool, route_task_tool, send_ack_tool]

print("🧰 Custom tools registered:", [t.name for t in custom_tools])


In [ ]:
# ======================================================================
# ⚙️ STEP 5 — Initialize Flotorch CrewAI Agent
# ======================================================================
# Connect to your configured agent in Flotorch Console.
# Make sure your Console agent has the correct role, goal, and backstory.
# ======================================================================

from flotorch.crewai.agent import FlotorchCrewAIAgent
from flotorch.crewai.sessions import FlotorchCrewAISession
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai import Crew


# Create Flotorch CrewAI agent client with custom tools
flotorch_client = FlotorchCrewAIAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools=custom_tools   # register your local action tools
)

# Get agent and task from Flotorch Console
agent = flotorch_client.get_agent()
task = flotorch_client.get_task()

print(f"✅ Agent tools attached: {[t.name for t in agent.tools]}")
print(f"✅ Agent has {len(agent.tools)} tool(s) available")

# Create Crew with agent, task, and memory
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=False
)

print("✅ Connected to Flotorch Agent:", agent.role)
response = crew.kickoff(inputs={
        "query": "Process unread mailbox: fetch and handle matching messages."
    })
response.raw if response else None



In [ ]:
# ======================================================================
# ▶️ Sample email examples for the agent to process
# ======================================================================
#sample1:Feature Request (UI)
'''
Subject: Feature: Add confirmation pop-up for experiment deletion

Body:
Hi Team,

When users delete an experiment there is no confirmation popup. Please implement a confirmation dialog on the UI to avoid accidental deletes.

Steps to reproduce:
1) Open experiment list.
2) Click delete on an experiment.
3) Observe deletion happens immediately.

Thanks,
QA
'''

#sample2:Bug Report (Backend)
'''
Subject: Bug: API returns 500 on /v1/experiments

Body:

Hi,

The experiments list API intermittently returns 500 errors. This started after the last deployment.

Error: InternalServerError stack trace...
Affects production for some users.

Please investigate urgently.
'''
